## backward propagation

##### 思路
1. 建立一个类似于pytorch的神经网络类
2. 验证反向传播值是否正确
3. 如何训练

### Implement via Pytorch

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

In [34]:
class SimpleNN(nn.Module):
    """docstring for SimpleNN."""
    def __init__(self, layers_dim):
        super(SimpleNN, self).__init__()
        layers = []
        for i in range(len(layers_dim) - 1):
            layers.append(nn.Linear(layers_dim[i], layers_dim[i+1]))
            layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers[:-1])        # Remove the last sigmoid layer
    
    def forward(self, x):
        return self.layers(x)

# test & validate
input_dim, output_dim = 10, 5
batch_size = 32
input_data = torch.randn(batch_size, input_dim)
target_data = torch.randn(batch_size, output_dim)

# initalized
layer_dims = [input_dim, 20, 15, output_dim]
model = SimpleNN(layer_dims)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# forward and backward propagation
optimizer.zero_grad()
output = model(input_data)
print(input_data.shape)
print(output.shape)
loss = criterion(output, target_data)
print(loss)
loss.backward()

# 查看梯度信息
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"Parameter: {name}, Gradient: {param.grad}")
        
optimizer.step()

torch.Size([32, 10])
torch.Size([32, 5])
tensor(0.9904, grad_fn=<MseLossBackward0>)
Parameter: layers.0.weight, Gradient: tensor([[ 1.3943e-04,  4.6572e-04, -3.3065e-04,  3.5841e-04,  1.5319e-04,
         -1.4903e-04, -6.0108e-04, -3.7794e-04,  1.4852e-04,  1.0288e-04],
        [-2.8646e-04,  2.4914e-04,  1.0915e-03,  4.0133e-07, -9.6309e-05,
         -1.1860e-03, -3.2893e-04, -5.4088e-04, -9.7305e-04,  4.5672e-04],
        [-3.1324e-04,  1.3100e-04, -5.3196e-04,  2.7500e-04, -6.6554e-05,
          2.8644e-04, -7.0296e-05, -2.5064e-04,  6.9971e-04, -3.4421e-04],
        [ 4.1571e-04, -4.3944e-05,  9.2465e-04, -3.3053e-04,  2.4716e-04,
         -6.8029e-04, -3.0840e-05,  3.1085e-04, -1.1051e-03,  6.4344e-04],
        [ 4.4178e-05,  7.7999e-06,  2.0557e-04,  2.9671e-04, -2.0258e-04,
         -4.4942e-04, -1.3752e-04, -5.1561e-04, -2.9037e-04, -1.9371e-04],
        [-2.2254e-04,  4.2136e-05, -2.7577e-04, -1.3525e-04, -1.5094e-05,
          2.4938e-04,  6.0952e-05,  1.1661e-04,  3.0963e-04

### Implement 1 from scratch
source from https://mp.weixin.qq.com/s/-3w3qZISIUb5OoshkCzOwQ

code in line 33 is wrong, should be `z_temp =parameters["w"+str(i)].dot(a[-1]) + parameters["b"+str(i)]`

尝试使用relu代替sigmoid，发现以下几点：
1. relu要使用在足够深的神经网络中
2. relu是稀疏激活的
3. 在实践中，relu 激活通常与正则化技术（例如 dropout 或批量归一化）结合使用
4. 使用大型数据集进行训练：GPT 和 Transformer 等模型通常在大型数据集上进行训练，这有助于弥补 relu 激活的潜在缺点。有了足够多的数据，该模型就能够学习稳健的表示并很好地泛化，即使负输入的 ReLU 问题即将解决。


In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# 生成权重以及偏执项layers_dim代表每层的神经元个数，
#比如[2,3,1]代表一个三成的网络，输入为2层，中间为3层输出为1层
def init_parameters(layers_dim):
    
    L = len(layers_dim)
    parameters ={}
    for i in range(1,L):
        parameters["w"+str(i)] = np.random.random([layers_dim[i],layers_dim[i-1]])
        parameters["b"+str(i)] = np.zeros((layers_dim[i],1))
    return parameters

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# sigmoid的导函数
def sigmoid_prime(z):
        return sigmoid(z) * (1-sigmoid(z))

# 前向传播，需要用到一个输入x以及所有的权重以及偏执项，都在parameters这个字典里面存储
# 最后返回会返回一个caches里面包含的 是各层的a和z，a[layers]就是最终的输出
def forward(x,parameters):
    a = []
    z = []
    caches = {}
    a.append(x)
    z.append(x)
    layers = len(parameters)//2
    # 前面都要用sigmoid
    for i in range(1,layers):
        z_temp =parameters["w"+str(i)].dot(a[-1]) + parameters["b"+str(i)]
        z.append(z_temp)
        a.append(sigmoid(z_temp))
    # 最后一层不用sigmoid
    z_temp = parameters["w"+str(layers)].dot(a[layers-1]) + parameters["b"+str(layers)]
    z.append(z_temp)
    a.append(z_temp)
    
    caches["z"] = z
    caches["a"] = a    
    return  caches,a[layers]

# 反向传播，parameters里面存储的是所有的各层的权重以及偏执，caches里面存储各层的a和z
# al是经过反向传播后最后一层的输出，y代表真实值
# 返回的grades代表着误差对所有的w以及b的导数
def backward(parameters,caches,al,y):
    layers = len(parameters)//2
    grades = {}
    m = y.shape[1]
    # 假设最后一层不经历激活函数
    # 就是按照上面的图片中的公式写的
    grades["dz"+str(layers)] = al - y
    grades["dw"+str(layers)] = grades["dz"+str(layers)].dot(caches["a"][layers-1].T) /m
    grades["db"+str(layers)] = np.sum(grades["dz"+str(layers)],axis = 1,keepdims = True) /m
    # 前面全部都是sigmoid激活
    for i in reversed(range(1,layers)):
        grades["dz"+str(i)] = parameters["w"+str(i+1)].T.dot(grades["dz"+str(i+1)]) * sigmoid_prime(caches["z"][i])
        grades["dw"+str(i)] = grades["dz"+str(i)].dot(caches["a"][i-1].T)/m
        grades["db"+str(i)] = np.sum(grades["dz"+str(i)],axis = 1,keepdims = True) /m
    return grades   

# 就是把其所有的权重以及偏执都更新一下
def update_grades(parameters,grades,learning_rate):
    layers = len(parameters)//2
    for i in range(1,layers+1):
        parameters["w"+str(i)] -= learning_rate * grades["dw"+str(i)]
        parameters["b"+str(i)] -= learning_rate * grades["db"+str(i)]
    return parameters
# 计算误差值
def compute_loss(al,y):
    return np.mean(np.square(al-y))

# 加载数据
def load_data():
    """
    加载数据集
    """
    x = np.arange(0.0,1.0,0.01)
    y =20* np.sin(2*np.pi*x)
    # 数据可视化
    plt.scatter(x,y)
    return x,y
#进行测试
x,y = load_data()
x = x.reshape(1,100)
y = y.reshape(1,100)
plt.scatter(x,y)
parameters = init_parameters([1,25,1])
al = 0
for i in range(4000):
    caches,al = forward(x, parameters)
    grades = backward(parameters, caches, al, y)
    parameters = update_grades(parameters, grades, learning_rate= 0.3)
    if i %100 ==0:
        print(compute_loss(al, y))
plt.scatter(x,al)
plt.show()

### Implement 2 from scratch
source from [Nielsen' book](https://mp.weixin.qq.com/s/-3w3qZISIUb5OoshkCzOwQ) and [this github repo](https://github.com/Owly-dabs/Matrix-based_backprop_over_mini-batch/blob/main/network_matx.py)

In [ ]:
#! ~/miniconda3/envs/nndl/bin/python python3.5

"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning algorithm for a feedforward neural network. Gradients are calculated using backpropagation.
Code is intended to be simple, easily readable, and easily modifiable. It is not optimized, and omits many desirable features.
"""

# import library
import numpy as np
import random

# Network class
class Network():

    def __init__(self,sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])]

    def feedforward(self,a):
        for b,w in zip(self.biases,self.weights):
            a = sigmoid(np.dot(w,a) + b)
        return a

    def SGD(self,training_data,epochs,mini_batch_size,eta,test_data=None):
        """
        Train the neural network using mini-batch stochastic gradient descent.
        The 'training_data' is a list of tuples (x,y) representing the 
        training inputs and the desired outputs.
        The other non-optional parameters are self-explantory. 

        If 'test_data' is provided then the network will be evaluated against 
        the test data after each epoch, and partial progress printed out.
        This is useful for tracking progress, but slows things down substantially.
        """
        training_data = list(training_data)
        n = len(training_data)

        if test_data: 
            test_data = list(test_data)
            n_test = len(test_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size] 
                for k in range(0,n,mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch,eta)
            if test_data:
                print("Epoch {}: {}/{}".format(j,self.evaluate(test_data),n_test))
            else:
                print("Epoch {} complete".format(j))
            
    def update_mini_batch(self,mini_batch,eta):
        """
        Update the network's weights and biases by applying 
        gradient descent using back propagation to a single mini batch.
        The 'mini_batch' is a list of tuples (x,y), and 'eta' is the learning rate
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x,y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb+dnb for nb,dnb in zip(nabla_b,delta_nabla_b)]
            nabla_w = [nw+dnw for nw,dnw in zip(nabla_w,delta_nabla_w)]
        
        self.weights = [w-(eta/len(mini_batch))*nw for w,nw in zip(self.weights,nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b,nb in zip(self.biases,nabla_b)]         

    def backprop(self,x,y):
        """
        Return a tuple `(nabla_b, nabla_w)` representing the gradient for the cost function C_x. 
        `nabla_b` and `nabla_w` are layer-by-layer lists of numpy arrays, similar to `self.biases` and `self.weights`.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # backward pass
        delta = self.cost_derivative(activations[-1],y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Note that the variable l in the loop below is used a little differently to the notation in Chapter 2 of the book. 
        # Here, l=1 means the last layer of neurons, l=2 is the second-last layer, and so on. 
        # It's a renumbering of the scheme in the book, used here to take advantage of the fact that python can use negative indices in lists.

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        
        return(nabla_b,nabla_w)

    def evaluate(self,test_data):
        """
        Return the number of test inputs for which the neural network outputs the correct result.
        Note that the neural network's output is assumed to be the index of whichever neuron in the final layer has the highest activation.
        """
        test_results = [(np.argmax(self.feedforward(x)),y) for (x,y) in test_data]
        return sum(int(x==y) for (x,y) in test_results)

    def cost_derivative(self, output_activations,y):
        """
        Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations
        """
        return (output_activations-y)


#Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """
    Derivative of the sigmoid function.
    """
    return sigmoid(z)*(1-sigmoid(z))

### Implement 2 from scratch -- matrix-based version
https://github.com/Owly-dabs/Matrix-based_backprop_over_mini-batch/blob/main/network.py

In [ ]:
import random
import numpy as np

class Network():
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])]
    
    def feedforward(self,a):
        for b,w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w,a) + b)
        return a

    # SGD
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data):
        # turn training_data from tuple into list
        training_data = list(training_data)
        n = len(training_data)
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for e in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)]
            for mini_batch in mini_batches:
                # x.shape = (number of examples in mini batch, (784, 1))
                self.update_mini_batch(mini_batch,eta)

            if test_data:
                print("Epoch {}: {}/{}".format(e,self.evaluate(test_data),n_test))
            else:
                print("Epoch {} complete.".format(e))


    # Update mini batch
    def update_mini_batch(self,mini_batch,eta):
        X = [x for x,y in mini_batch]
        Y = [y for x,y in mini_batch]
        nabla_b, nabla_w = self.backprop(X,Y)
        
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w,nw in zip(self.weights,nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                        for b,nb in zip(self.biases, nabla_b)]
       

    # backprop
    def backprop(self, X, Y):
        """
        Return a tuple `(nabla_b, nabla_w)` representing the gradient
        for the cost_function C_x.
        `nabla_b` and `nabla_w` are layer-by-layer lists of numpy arrays,
        similar to `self.biases` and `self.weights`
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # feedforward
        activation = X # shape = (mini_batch_size, 784, 1) 
        activations = [X] # list to store all activations, layer-by-layer
        Zs = [] # list to store all z vectors, layer-by-layer
        for b,w in zip(self.biases,self.weights):
            Z = np.transpose(np.dot(w, activation),(1,0,2)) + b
            Zs.append(Z)
            activation = sigmoid(Z)
            activations.append(activation)
        
        # backward pass
        delta_batch = self.cost_derivative(activations[-1],Y) * \
            sigmoid_prime(Zs[-1]) # shape = (mini_batch_size,10,1)

        ##TODO: figure out how to get delta to fit into nabla and 
        # match shapes with self.weights and self.biases
        # Refer to shell 
        nabla_b[-1] = np.sum(delta_batch, axis=0)
        nabla_w[-1] = np.squeeze(np.dot(np.transpose(delta_batch,(2,1,0)),np.transpose(activations[-2],(2,0,1))))

        for l in range(2, self.num_layers):
            Z = Zs[-l]
            sp = sigmoid_prime(Z)
            delta_batch = np.transpose(np.dot(self.weights[-l+1].transpose(), delta_batch),(1,0,2)) * sp
            nabla_b[-l] = np.sum(delta_batch,axis=0)
            nabla_w[-l] = np.squeeze(np.dot(np.transpose(delta_batch,(2,1,0)), np.transpose(activations[-l-1],(2,0,1))))
        
        return(nabla_b,nabla_w)
    def evaluate(self,test_data):
        """
        Return the number of test inputs for which the neural network outputs the correct result.
        Note that the neural network's output is assumed to be the index of whichever neuron in the final layer has the highest activation.
        """
        test_results = [(np.argmax(self.feedforward(x)),y) for (x,y) in test_data]
        return sum(int(x==y) for (x,y) in test_results)
        
    def cost_derivative(self,output_activations,y):
        """
        Return the vector of partial derivatives (partial C_X/partial A)
        for the output activations.
        """
        return (output_activations-y)
        


def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))